## GOLD TO API LAYER

### API Layer


In [92]:
# Importing Common Utility Function
import json
import pandas as pd
from StockETL import DateTimeUtil, GlobalPath

In [93]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [94]:
# Define file paths using GlobalPath
current_holding_records_file_path = GlobalPath(
    "DATA/GOLD/Holding/CurrentHolding_data.csv"
)
stockprice_silver_file_path = GlobalPath("DATA/SILVER/StockPrice/StockPrice_data.csv")
holding_gold_file_path = GlobalPath("DATA/GOLD/Holding/Holding_data.csv")
profitloss_gold_file_path = GlobalPath("DATA/GOLD/ProfitLoss/ProfitLoss_data.csv")
dividend_gold_file_path = GlobalPath("DATA/GOLD/Dividend/Dividend_data.csv")

In [95]:
def group_holding_data(df_current_holding):
    """Groups holding data by scrip_name, symbol, exchange, and segment to calculate totals and min datetime."""
    df_current_holding["datetime"] = pd.to_datetime(df_current_holding["datetime"])
    df_current_holding = (
        df_current_holding.groupby(["scrip_name", "symbol", "exchange", "segment"])
        .agg(
            total_quantity=("quantity", "sum"),
            total_amount=("amount", "sum"),
            min_datetime=("datetime", "min"),
        )
        .reset_index()
    )
    df_current_holding["avg_price"] = (
        df_current_holding["total_amount"] / df_current_holding["total_quantity"]
    )
    return df_current_holding

In [96]:
def process_stock_prices(df_stockprice):
    """Processes stock prices to extract the latest closing price for each symbol."""
    df_stockprice["date"] = pd.to_datetime(df_stockprice["date"])
    df_stockprice["close_price"] = df_stockprice["close"]
    idx = df_stockprice.groupby("symbol")["date"].idxmax()
    return df_stockprice.loc[idx].reset_index(drop=True)

In [97]:
def merge_and_calculate_pnl(df_current_holding, df_stockprice):
    """Merges holding data with stock prices and calculates PnL."""
    df_current_holding = pd.merge(
        df_current_holding,
        df_stockprice[["symbol", "close_price"]],
        on="symbol",
        how="left",
    )
    df_current_holding["close_amount"] = (
        df_current_holding["close_price"] * df_current_holding["total_quantity"]
    )
    df_current_holding["pnl_amount"] = (
        df_current_holding["close_amount"] - df_current_holding["total_amount"]
    )
    return df_current_holding.round(2)

In [98]:
def prepare_current_holding_data(df_current_holding_grouped, df_current_holding):
    """Prepares the current holding data with historical records."""
    current_holding = []
    for _, row in df_current_holding_grouped.iterrows():
        row = row.to_dict()
        df_filtered = df_current_holding[
            (df_current_holding["scrip_name"] == row["scrip_name"])
            & (df_current_holding["symbol"] == row["symbol"])
            & (df_current_holding["exchange"] == row["exchange"])
            & (df_current_holding["segment"] == row["segment"])
        ]
        row["history"] = df_filtered.to_dict(orient="records")
        current_holding.append(row)
    return current_holding

In [99]:
# Main flow
df_current_holding = pd.read_csv(current_holding_records_file_path)
print(f"Processing Data : {current_holding_records_file_path}")

df_stockprice = pd.read_csv(stockprice_silver_file_path)
print(f"Processing Data : {stockprice_silver_file_path}")

df_stockprice = process_stock_prices(df_stockprice)
df_current_holding_grouped = group_holding_data(df_current_holding)
df_current_holding_grouped = merge_and_calculate_pnl(
    df_current_holding_grouped, df_stockprice
)
current_holding = prepare_current_holding_data(
    df_current_holding_grouped, df_current_holding
)

Processing Data : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\Holding\CurrentHolding_data.csv
Processing Data : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SILVER\StockPrice\StockPrice_data.csv


In [100]:
df_holding = pd.read_csv(holding_gold_file_path)
print(f"Processing Data : {holding_gold_file_path}")

# Processes holding history data to calculate trends
df_holding["date"] = pd.to_datetime(df_holding["date"]).dt.date
df_holding = (
    df_holding.groupby("date")[
        [
            "holding_amount",
            "open_amount",
            "high_amount",
            "low_amount",
            "close_amount",
        ]
    ]
    .sum()
    .reset_index()
)

df_holding = df_holding.round(2).rename(
    columns={col: col.replace("_amount", "") for col in df_holding.columns}
)

df_holding = (
    df_holding[["date", "open", "high", "low", "close", "holding"]]
    .sort_values(by=["date"])
    .reset_index(drop=True)
)

Processing Data : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\Holding\Holding_data.csv


In [101]:
df_pnl = pd.read_csv(profitloss_gold_file_path)
print(f"Processing Data : {profitloss_gold_file_path}")

Processing Data : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv


In [102]:
df_dividend = pd.read_csv(dividend_gold_file_path)
print(f"Processing Data : {dividend_gold_file_path}")

Processing Data : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\Dividend\Dividend_data.csv


In [103]:
# Create the final output dictionary for the API
load_timestamp = DateTimeUtil.today()
api_outputs = {
    "current_holding_data": current_holding,
    "holding_trands_data": df_holding.to_dict(orient="records"),
    "profit_loss_data": df_pnl.to_dict(orient="records"),
    "dividend_data": df_dividend.to_dict(orient="records"),
}

# CLEAN UP EXISTING DATA BEFORE RELOAD
GlobalPath.del_dir(GlobalPath("DATA/API/"))

for api_name, output in api_outputs.items():
    api_file_path = GlobalPath(f"DATA/API/{api_name}.json")
    # Clean the output
    cleaned_output = replace_nan_with_empty(output)

    # Write the result to a JSON file
    with open(api_file_path, "w", encoding="utf-8") as json_file:
        json.dump(
            {"load_timestamp": load_timestamp, "data": cleaned_output},
            json_file,
            indent=4,
            default=str,
        )

    print(f"Data written to {api_file_path}")

Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\API\current_holding_data.json
Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\API\holding_trands_data.json
Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\API\profit_loss_data.json
Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\API\dividend_data.json
